In [33]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [34]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-10-02 00:31:02--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.2’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2021-10-02 00:31:02 (6.62 MB/s) - ‘postgresql-42.2.16.jar.2’ saved [1002883/1002883]



In [35]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [36]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_02.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Books_v1_02.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12076615| RQ58W7SMO911M|0385730586|     122662979|Sisterhood of the...|           Books|          4|            2|          3|   N|                N|this book was a g...|this boook was a ...| 2005-10-14|
|         US|   12703090|  RF6IUKMGL8SF|0811828964|      56191234|The Bad Girl's Gu...|           Books|          3|    

In [37]:
df.groupBy("vine").count().show()

+----+-------+
|vine|  count|
+----+-------+
|null|      5|
|   Y|      2|
|   N|3105513|
+----+-------+



In [38]:
# Create the vine_table. DataFrame
vine_df = df.select(["total_votes", "review_id", "star_rating", "helpful_votes", "vine", "verified_purchase"])
vine_df.show(5)

+-----------+--------------+-----------+-------------+----+-----------------+
|total_votes|     review_id|star_rating|helpful_votes|vine|verified_purchase|
+-----------+--------------+-----------+-------------+----+-----------------+
|          3| RQ58W7SMO911M|          4|            2|   N|                N|
|          5|  RF6IUKMGL8SF|          3|            5|   N|                N|
|         22|R1DOSHH6AI622S|          4|            1|   N|                N|
|          2| RATOTLA3OF70O|          5|            2|   N|                N|
|          2|R1TNWRKIVHVYOV|          4|            0|   N|                N|
+-----------+--------------+-----------+-------------+----+-----------------+
only showing top 5 rows



In [40]:
#Step 1
#Create df of total vote count is greater than or equal to 20
votes_df = vine_df.filter(vine_df.total_votes >= 20)
votes_df.show(5)

+-----------+--------------+-----------+-------------+----+-----------------+
|total_votes|     review_id|star_rating|helpful_votes|vine|verified_purchase|
+-----------+--------------+-----------+-------------+----+-----------------+
|         22|R1DOSHH6AI622S|          4|            1|   N|                N|
|         20|R1VE0FQQ0QTQJN|          5|           16|   N|                Y|
|         35|R1AABFZH0J0C0C|          3|           17|   N|                N|
|         34|R3SJYLG07EHNE9|          2|           16|   N|                N|
|         31|R3MD5TZJGIPTDU|          5|           21|   N|                N|
+-----------+--------------+-----------+-------------+----+-----------------+
only showing top 5 rows



In [41]:
#Step 2
#create new df based on last df filtered on helpful votes / total votes greater than or equal to 50%
ratio_df = votes_df.filter((votes_df.helpful_votes / votes_df.total_votes) >= 0.5)
ratio_df.show(20)

+-----------+--------------+-----------+-------------+----+-----------------+
|total_votes|     review_id|star_rating|helpful_votes|vine|verified_purchase|
+-----------+--------------+-----------+-------------+----+-----------------+
|         20|R1VE0FQQ0QTQJN|          5|           16|   N|                Y|
|         31|R3MD5TZJGIPTDU|          5|           21|   N|                N|
|         28| RX6OWUF8EFZ72|          5|           27|   N|                N|
|         27|R1OPUF771LL5O5|          4|           27|   N|                N|
|        307|R3AW5JRZYH66M5|          1|          268|   N|                N|
|         27|R2DXLNKF0EL4CI|          4|           24|   N|                Y|
|         24|R12EHXBC0GP2C2|          5|           23|   N|                N|
|        184|R2JSWNTAO83X3V|          5|          145|   N|                N|
|         47|R3THZWOBS1086J|          5|           46|   N|                N|
|         30|R160DHJPJ4B20R|          4|           28|   N|     

In [50]:
#create another df where reviews came from vine
vine_yes_df = vine_df.filter(vine_df.vine == 'Y')
vine_yes_df.show(5)

+-----------+--------------+-----------+-------------+----+-----------------+
|total_votes|     review_id|star_rating|helpful_votes|vine|verified_purchase|
+-----------+--------------+-----------+-------------+----+-----------------+
|          3|R2NKNL4WXWIWBG|          4|            3|   Y|                Y|
|          6| RAIG9ZQDMBF5P|          5|            3|   Y|                N|
+-----------+--------------+-----------+-------------+----+-----------------+



In [51]:
#create another df where reviews came from vine
vine_no_df = ratio_df.filter(ratio_df.vine == 'N')
vine_no_df.show(5)

+-----------+--------------+-----------+-------------+----+-----------------+
|total_votes|     review_id|star_rating|helpful_votes|vine|verified_purchase|
+-----------+--------------+-----------+-------------+----+-----------------+
|         20|R1VE0FQQ0QTQJN|          5|           16|   N|                Y|
|         31|R3MD5TZJGIPTDU|          5|           21|   N|                N|
|         28| RX6OWUF8EFZ72|          5|           27|   N|                N|
|         27|R1OPUF771LL5O5|          4|           27|   N|                N|
|        307|R3AW5JRZYH66M5|          1|          268|   N|                N|
+-----------+--------------+-----------+-------------+----+-----------------+
only showing top 5 rows



In [52]:
total_paid_reviews = vine_yes_df.count()
total_paid_reviews

2

In [53]:
paid_five_star_reviews = vine_yes_df.filter(vine_yes_df.star_rating == 5).count()
paid_five_star_reviews

1

In [54]:
paid_five_star_percent = (paid_five_star_reviews / total_paid_reviews) * 100
paid_five_star_percent

50.0

In [55]:
total_unpaid_reviews = vine_no_df.count()
total_unpaid_reviews

403807

In [56]:
unpaid_five_star_reviews = vine_no_df.filter(vine_no_df.star_rating == 5).count()
unpaid_five_star_reviews

242889

In [57]:
unpaid_five_star_percent = (unpaid_five_star_reviews / total_unpaid_reviews) * 100
unpaid_five_star_percent

60.14977452099642